In [ ]:
# only for google drive
!pip install git+https://github.com/seankmartin/simuran.git@main
!pip install allensdk==2.14.1
!pip install one-api==1.17.0

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import simuran
from simuran.loaders.allen_loader import AllenVisualBehaviorLoader
# from simuran.loaders.one_loader import OneAlyxLoader

In [ ]:
ALLEN_CACHE_DIR = Path(r"D:\example-data\allen-data")
ALLEN_MANIFEST = "visual-behavior-neuropixels_project_manifest_v0.3.0.json"
IBL_CACHE_DIR = Path(r"D:\example-data\ibl-data")

In [ ]:
# loader = AllenVisualBehaviorLoader(cache_directory=ALLEN_CACHE_DIR, manifest=ALLEN_MANIFEST)
# loader.create_s3_cache()
# loader.cache.compare_manifests(ALLEN_MANIFEST, ALLEN_MANIFEST.replace("3", "4"))

In [ ]:
def create_allen_recording_container(cache_directory, manifest=None) -> "simuran.RecordingContainer":
    loader = AllenVisualBehaviorLoader(cache_directory=cache_directory, manifest=manifest)
    loader.create_s3_cache()
    ecephys_sessions = loader.cache.get_ecephys_session_table()
    sessions = ecephys_sessions.dropna(subset=["file_id"])
    recording_container = simuran.RecordingContainer.from_table(ecephys_sessions, loader)
    return recording_container

In [ ]:
rc = create_allen_recording_container(ALLEN_CACHE_DIR, ALLEN_MANIFEST)
for r in rc:
    if r.attrs["dowloaded"]:
        print(f"{r.source_file} is downloaded")
        print(r)
    else:
        print(f"{r.source_file} is not yet available")